In [14]:
import numpy as np
import time

def thermodynamic_constants(compounds):
    properties = {
        "water": {"antoine": [5.11564, 1687.537, 230.17], 
                   "Cp_liq": 76.392/1000, "Cp_vap": 37.527/1000, "Hvap": 37.527, "activity coefficient":1}, #Hvap is at Tref
        "nitric acid": {"antoine": [4.28523, 1337.035, 224.78], 
                         "Cp_liq": 110.679/1000, "Cp_vap": 53.336/1000, "Hvap": 28.387, "activity coefficient":0.725}, #Hvap is at Tref
        "ammonium nitrate": {"antoine": [7.105, 4360, 0], 
                         "Cp_liq": 132.2144/1000, "Cp_vap": 85.353/1000, "Hvap": 50, "activity coefficient":0.142} #Hvap is at Tb: 356.15 K
        
    }
        
    cts_antoine = np.array([properties[comp]["antoine"] for comp in compounds])
    Cp_liq = np.array([properties[comp]["Cp_liq"] for comp in compounds])
    Cp_vap = np.array([properties[comp]["Cp_vap"] for comp in compounds])
    Hvap = np.array([properties[comp]["Hvap"] for comp in compounds])
    Activity_coefficient = np.array([properties[comp]["activity coefficient"] for comp in compounds])
 
    return (cts_antoine, Cp_liq, Cp_vap, Hvap, Activity_coefficient)


In [15]:
def pure_vapor_pressure(compounds, T):
    
    cts, _ , _, _ , _ = thermodynamic_constants(compounds)
    p_vap = np.zeros(len(compounds))
    for i in range(len(compounds)):
        A = cts[i][0]
        B = cts[i][1]
        C = cts[i][2]
        
        exp_vap = A - (B / (T + C - 273.15)) #antoine equation
        p_vap[i] = pow(10, exp_vap)
        
    return p_vap

print (pure_vapor_pressure(["nitric acid"],356.15))

[0.8731811]


In [24]:

def adiabatic_flash(compounds, T_feed, P_op, z_feed, F):
    
    cts, Cp_liq, Cp_vap, Hvap, activity_coefficients = thermodynamic_constants(compounds)
    
    Energy_tol = 1e-3 #change to 1e-3, 1e-4 or 1e-5 for later iterations and increased accuracy
    PSI_tol = 1e-10
    T_ref = 298.15 #K
    r = 100000
    
    # Calculating feed enthalpy h_f
    h_f = sum(z_feed * Cp_liq * (T_feed - T_ref))
    
    # Initial guesses for T_drum (use two values for Secant method)
    T_drum = T_feed - 30 #K
    T_drum_prev = T_drum + 1  # Slightly higher than the first guess

    # Compute initial energy balance values
    Energy_balance_old = 1e6  # Initialize with a large number
    Energy_balance_prev = Energy_balance_old  # Set prev energy balance

    
    #----------------------------------------------------------------------------------
    # LOOP TO SOLVE ENERGY AND MASS BALANCE SIMULTANIOUSLY:
    iteration = 0
    max_iterations = 5000000 #a limit so it doesn't take forever. 5M iterations take ~15 mins
    
    Start_Time = time.time()
    
    while abs(Energy_balance_old) > Energy_tol and iteration < max_iterations:
        #equilibirum relations to get x, y, L, V and psi (V/F)
        #Rachfor-Rice for isothermal flash
        Pi_sat = pure_vapor_pressure(compounds, T_drum) #saturation pressure
        Ki = (activity_coefficients * Pi_sat) / P_op  #modified raoults law

        psi_o = min(0.4, sum(z_feed * Ki) / sum(Ki)) # initial guess for the phase fraction (psi)
        xo = []

        fob_rr = 1.0
        
        count = 0
        while abs(fob_rr) > PSI_tol and count < 1000:

            num_rr = z_feed * (1 - Ki)
            denom_rr = 1 + psi_o * (Ki - 1)
            fob_rr = sum(num_rr / denom_rr)

            xo = z_feed / (1.0 + (psi_o * (Ki - 1.0)))

            V = psi_o * F
            L = F - V

            yo = Ki * xo

            #Newton-Rhapson to get new guess for psi (vapour phase fraction)
            defo_rr = sum((z_feed * ((Ki - 1.0) ** 2)) / ((1.0 + (psi_o * (Ki - 1.0))) ** 2))
            psi_1 = psi_o - np.sign(fob_rr) * min(abs(fob_rr / (defo_rr + 1e-8)), 0.1)
            psi_o = psi_1
            psi_o = max(0.001, min(psi_o, 0.999))
            
            count += 1
            
        if count >= 1000:
            print("WARNING: PSI did not converge, iteration number:", iteration)
            print(" ")
            #If iteration number of this is the same as the iteration number for answers, discard answers

        PSI = psi_o

        #Calculate liquid and vapour enthalpies at guessed T_drum
        h_L = sum(xo * Cp_liq * (T_drum - T_ref))
        H_V = sum(yo * (Hvap + Cp_vap * (T_drum - T_ref)))

        
        #Compute new energy balance
        Energy_balance_old = F * h_f - (V * H_V + L * h_L)

        # **Use the Secant Method to update T_drum**
        if abs(Energy_balance_old) > Energy_tol:
            
            if iteration < 2:
                T_drum_new = T_drum - 2  # Small initial step change
            else:
                T_drum_new = max(273, min(450, T_drum + 0.5 * np.sign(Energy_balance_old) * 
                       min(abs(Energy_balance_old / (Energy_balance_old - Energy_balance_prev + 1e-8)), 5)))
                
            #Update temperature values for next iteration
            T_drum_prev = T_drum
            Energy_balance_prev = Energy_balance_old
            T_drum = T_drum_new
        
        #Check if there is anything wrong with the iterations
        if iteration %r == 0:
            print("iteration number", iteration)
            End_Time = time.time()
            Time_Taken = (End_Time - Start_Time)/60
            print(f"Time taken: {Time_Taken:.2f} min")
            print("Equilibrium Constants (Ki):", Ki)
            print(f"T_drum = {T_drum:.2f}, Energy Balance = {Energy_balance_old:.2f}, PSI = {PSI:.4f}")
            print(" ")
            
        iteration += 1
        
    
    if iteration >= max_iterations:
        #Change the Energy_tol to a bigger number
        print("-------------------------------------------------------------------------------------------")
        print("WARNING: main loop did not converge")
        print(" ")
        print(f"T_drum = {T_drum:.2f}, Energy Balance = {Energy_balance_old:.2f}, PSI = {PSI:.4f}")
        print("-------------------------------------------------------------------------------------------")
    
    #See how long the iterations took
    print("iteration number", iteration)
    End_Time = time.time()
    Time_Taken = (End_Time - Start_Time)/60
    print(f"Time taken: {Time_Taken:.2f} min")
    print(" ")
    print("remaining energy", Energy_balance_old, "kJ kmol / mol h")
    print(" ")
    
    return PSI, V, L, xo, yo, T_drum

In [25]:
compounds = ["water", "nitric acid","ammonium nitrate"]
T_feed = 149.8 + 273.15  # K
P_op = 0.35  # bar
z_feed = np.array([0.28055163945180600,0.00432559380666241,0.71512276674153200])  # molar feed composition
F = 139.10877379961100000 # kmol/h should be mol/h when i checked units

PSI, V, L, x_liq, y_vap, T_drum = adiabatic_flash(compounds, T_feed, P_op, z_feed, F)

print("Adiabatic Flash Separation Results:")
print("Vapor Fraction (PSI):", PSI)
print("Vapor Flow Rate (kmol/h):", V)
print("Liquid Flow Rate (kmol/h):", L)
print("Liquid Composition:", x_liq)
print("Vapor Composition:", y_vap)
print("Temperature of Flash Drum:", T_drum)

iteration number 0
Time taken: 0.00 min
Equilibrium Constants (Ki): [5.61855142e+00 5.26395887e+00 2.08559663e-30]
T_drum = 390.95, Energy Balance = -122425.62, PSI = 0.1299
 
iteration number 100000
Time taken: 0.33 min
Equilibrium Constants (Ki): [5.13641616e+00 4.89174521e+00 2.79625547e-31]
T_drum = 390.33, Energy Balance = 5762.96, PSI = 0.1118
 
iteration number 200000
Time taken: 0.66 min
Equilibrium Constants (Ki): [5.17450231e+00 4.92137672e+00 3.30728232e-31]
T_drum = 390.10, Energy Balance = -5213.75, PSI = 0.1134
 
iteration number 300000
Time taken: 1.02 min
Equilibrium Constants (Ki): [5.14763923e+00 4.90048112e+00 2.93854802e-31]
T_drum = 390.53, Energy Balance = 2511.61, PSI = 0.1123
 
iteration number 400000
Time taken: 1.42 min
Equilibrium Constants (Ki): [5.12974908e+00 4.88655396e+00 2.71482194e-31]
T_drum = 390.32, Energy Balance = 7701.13, PSI = 0.1116
 
iteration number 500000
Time taken: 1.83 min
Equilibrium Constants (Ki): [5.13343860e+00 4.88942692e+00 2.75960

KeyboardInterrupt: 